<center><a href="New York City"><img src = "https://cdn.pixabay.com/photo/2016/07/31/09/53/munich-1558705_960_720.jpg" width = 800> </a>

<h1 align=center><font size = 5>The Battle Of Neighborhoods</font></h1>
<h1 align=center><font size = 5>Airbnb price prediction in Munich</h1>
<h1 align=center><font size = 4>by Selmir Kalender</font></h1>

## About
Over the course of two weeks, I had the opportunity to be as creative as I wanted to be and come up with an idea to leverage the Foursquare location data to explore or compare neighborhoods or cities of my choice or to come up with a problem where I could use the Foursquare location data to solve the problem.
No matter what I've decided to do, I had to make sure to provide sufficient justification of why I thought what I've wanted to do or solve was important and why would a client or a group of people be interested in my project.

<H2>Table of Contents</H2>
1. <a href="#item1">Introduction</a><br>
2. <a href="#item2">Business Background</a><br>  
3. <a href="#item3">Target Audience</a><br>  
4. <a href="#item4">Data Overview</a><br>  
5. <a href="#item5">Methodology</a><br> 
6. <a href="#item6">Results</a><br>
7. <a href="#item7">Conclusion</a><br>
8. <a href="#item8">References</a><br>


## 1. Introduction

Airbnb is a internet marketplace for short-term home and apartment rentals. It allows you to, for example, rent out your home for a week while you’re away, or rent out your empty bedroom. One challenge that Airbnb hosts face is determining the optimal nightly rent price. In many areas, renters are presented with a good selection of listings and can filter by criteria like price, number of bedrooms, room type, and more. Since Airbnb is a market, the amount a host can charge is ultimately tied to market prices.

## 2. Business Background

Although Airbnb provides hosts with general guidance, there are no easy to access methods to determine the best price to rent out a space. There is third-party software available, but for a hefty price (see an example on available software, click [here](https://beyondpricing.com/)).

One method could be to find a few listings that are similar to the place that will be up for rent, average the listed prices and set our price to this calculated average price. However, with the market being so dynamic, we would probably be looking to update the price regularly and this method can become tedious.

Moreover, this may not be very accurate, as we are not taking into account other important factors that may give us a comparative advantage over other listings around us. This could be property characteristics such as number of rooms, bathrooms and extra services on offer.

The aim of this project is to propose a data-driven solution, by using machine learning to predict rental price.

For this project, a predictor based on space will be introduced to the model: the property's proximity to certain venues. This will allow the model to put an implicit price on things such as living close to a bar or a supermarket.

## 3. Target Audience
Who will be beneficial from this project:
* *Hosts* who are interested to identify the market value for their apartments or rooms
* *Tourists* who would want to compare different hosts to find the best solution for themselves
* *Data Analysts/Scientists* who can view the analysis for this project and can get inspired to implement in other different problems using the techniques and ideas used here.

## 4. Data Overview
Airbnb does not release any data on the listings in its marketplace, a but separate group named [Inside Airbnb](http://insideairbnb.com/get-the-data.html) scrapes and compiles publicly available information about many cities Airbnb's listings from the Airbnb web-site. For this project, their data set scraped on May 24, 2020, on the city of Munich, Germany, is used. It contains information on all Munich Airbnb listings that were live on the site on that date (over 9,000). Here's a direct [link](http://insideairbnb.com/munich/).

The data has certain limitations. The most noticeable one is that it scrapes the advertised price rather than the actual price paid by previous customers. More accurate data is available for a fee in sites like [AirDNA](https://www.airdna.co/).

Each row in the dataset is a listing available for rental in Airbnb's site for the specific city (observations). The columns describe different characteristics of each listing (features).

Some of the more important features this project will look into are the following:

* accommodates: the number of guests the rental can accommodate
* bedrooms: number of bedrooms included in the rental
* bathrooms: number of bathrooms included in the rental
* beds: number of beds included in the rental
* price: nightly price for the rental
* minimum_nights: minimum number of nights a guest can stay for the rental
* maximum_nights: maximum number of nights a guest can stay for the rental
* number_of_reviews: number of reviews that previous guests have left

To model the spatial relationship between Airbnb rental prices and property proximity to certain venues, we use the [Foursquare API](https://developer.foursquare.com/) to access the city's venues and the street network.